In [1]:
import os
import pandas as pd
import tarfile
import yaml
import json

In [2]:
remediated_charts_dir = "RemediatedCharts"
subdirectory = "12factor"
file_path = remediated_charts_dir + "/" + subdirectory + "/values.yaml"

In [5]:
#Load values.yaml file
def load_yaml(file_path):
    with open(file_path, 'r') as file:
        data = yaml.safe_load(file)
    return data

data = load_yaml(file_path)
print(data)

{'nameOverride': '', 'fullnameOverride': '', 'image': {'repository': 'nginx', 'tag': '', 'pullPolicy': 'IfNotPresent'}, 'imagePullSecrets': [], 'serviceAccount': {'create': True, 'annotations': {}, 'name': ''}, 'hooks': [], 'deployment': {'replicaCount': 1, 'autoscaling': {'enabled': False, 'minReplicas': 1, 'maxReplicas': 100, 'targetCPUUtilizationPercentage': None, 'targetMemoryUtilizationPercentage': None, 'metrics': []}, 'annotations': {}, 'imagePullSecrets': [], 'volumes': [], 'application': {'securityContext': {}, 'env': None, 'envFrom': [], 'volumeMounts': [], 'command': [], 'args': [], 'ports': [], 'checks': {'startup': {}, 'liveness': {}, 'readiness': {}}, 'resources': {}}, 'sidecars': [], 'tasks': [], 'nodeSelector': {}, 'tolerations': [], 'affinity': {}, 'topologySpreadConstraints': []}, 'networking': {'service': {'annotations': {}, 'type': 'ClusterIP', 'clusterIP': '', 'ports': []}, 'ingress': {'enabled': False, 'className': '', 'annotations': {}, 'portName': 'http', 'hosts

In [26]:
#security_context_vals = next(find_nested_key(data,'securityContext'))
#print(security_context_vals)

{}


In [17]:
def find_nested_key(data, key_to_find, parent_key=None):
    if isinstance(data, dict):
        for key, value in data.items():
            if key == key_to_find:
                yield parent_key, key, value
            elif isinstance(value, (dict, list)):
                yield from find_nested_key(value, key_to_find, key)
    elif isinstance(data, list):
        for item in data:
            yield from find_nested_key(item, key_to_find, parent_key)


In [18]:
with open('nested_resolutions.json', 'r') as f:
    remediation_file = json.load(f)

print(remediation_file)
security_context_vals = list(find_nested_key(remediation_file, 'securityContext'))

print("All securityContext values found:")
for idx, sc in enumerate(security_context_vals, 1):
    print(f"{idx}: {sc}")

{'securityContext': {'allowPrivilegeEscalation': False, 'capabilities': {'drop': ['ALL'], 'add': []}, 'runAsNonRoot': True, 'readOnlyRootFilesystem': True, 'runAsUser': 10001, 'runAsGroup': 10001, 'seLinuxOptions': None, 'seccompProfile': {'type': 'RuntimeDefault'}}, 'resources': {'limits': {'cpu': '500m', 'memory': '1Gi'}, 'requests': {'cpu': '200m', 'memory': '512Mi'}}, 'hostNetwork': False, 'hostPID': False, 'hostIPC': False, 'volumes': [{'hostPath': None}], 'externalIPs': None, 'podAnnotations': {'container.apparmor.security.beta.kubernetes.io': 'runtime/default'}, 'rules': {'resources': [], 'verbs': ['list', 'watch', 'get']}, 'serviceAccount': {'anonymous-auth': False}, 'clusterRole': {'secrets': False, 'wildcard': False}, 'image': {'pullPolicy': 'IfNotPresent', 'tag': '1.0.0'}}
All securityContext values found:
1: (None, 'securityContext', {'allowPrivilegeEscalation': False, 'capabilities': {'drop': ['ALL'], 'add': []}, 'runAsNonRoot': True, 'readOnlyRootFilesystem': True, 'runAs

In [19]:
# Identify the sections we're interested in (e.g., SecurityContext, Resources)
def identify_sections(remediation_file):
    sections = {
        "SecurityContext": list(find_nested_key(remediation_file, "securityContext")),
        "Resources": list(find_nested_key(remediation_file, "resources")),
        "PodAnnotations": list(find_nested_key(remediation_file, "podAnnotations")),
        "Volumes": list(find_nested_key(remediation_file, "volumes")),
        "ExternalIPs": list(find_nested_key(remediation_file, "externalIPs")),
        "ServiceAccount": list(find_nested_key(remediation_file, "serviceAccount")),
        "ClusterRole": list(find_nested_key(remediation_file, "clusterRole")),
        "Rules": list(find_nested_key(remediation_file, "rules")),
        "Image": list(find_nested_key(remediation_file, "image")),
        "HostSettings": [
            *list(find_nested_key(remediation_file, "hostNetwork")),
            *list(find_nested_key(remediation_file, "hostPID")),
            *list(find_nested_key(remediation_file, "hostIPC")),
        ],
    }
    return sections

# Print selected sections
def print_sections(remediation_file):
    sections = identify_sections(remediation_file)
    print("Identified Sections:")
    for section, occurrences in sections.items():
        print(f"\nSection: {section}")
        if occurrences:
            for occurrence in occurrences:
                print(f"- {occurrence}")
        else:
            print("No occurrences found.")

In [20]:
# Identify the sections we're interested in (e.g., SecurityContext, Resources)
def identify_sections(data):
    sections = {
        "SecurityContext": list(find_nested_key(data, "securityContext")),
        "Resources": list(find_nested_key(data, "resources")),
        "PodAnnotations": list(find_nested_key(data, "podAnnotations")),
        "Volumes": list(find_nested_key(data, "volumes")),
        "ExternalIPs": list(find_nested_key(data, "externalIPs")),
        "ServiceAccount": list(find_nested_key(data, "serviceAccount")),
        "ClusterRole": list(find_nested_key(data, "clusterRole")),
        "Rules": list(find_nested_key(data, "rules")),
        "Image": list(find_nested_key(data, "image")),
        "HostSettings": [
            *list(find_nested_key(data, "hostNetwork")),
            *list(find_nested_key(data, "hostPID")),
            *list(find_nested_key(data, "hostIPC")),
        ],
    }
    return sections

# Print selected sections
def print_sections(data):
    sections = identify_sections(data)
    print("Identified Sections:")
    for section, occurrences in sections.items():
        print(f"\nSection: {section}")
        if occurrences:
            for occurrence in occurrences:
                print(f"- {occurrence}")
        else:
            print("No occurrences found.")

In [21]:
print_sections(data)

Identified Sections:

Section: SecurityContext
- ('application', 'securityContext', {})

Section: Resources
- ('application', 'resources', {})

Section: PodAnnotations
No occurrences found.

Section: Volumes
- ('deployment', 'volumes', [])

Section: ExternalIPs
No occurrences found.

Section: ServiceAccount
- (None, 'serviceAccount', {'create': True, 'annotations': {}, 'name': ''})

Section: ClusterRole
No occurrences found.

Section: Rules
No occurrences found.

Section: Image
- (None, 'image', {'repository': 'nginx', 'tag': '', 'pullPolicy': 'IfNotPresent'})

Section: HostSettings
No occurrences found.


In [22]:
print_sections(remediation_file)

Identified Sections:

Section: SecurityContext
- (None, 'securityContext', {'allowPrivilegeEscalation': False, 'capabilities': {'drop': ['ALL'], 'add': []}, 'runAsNonRoot': True, 'readOnlyRootFilesystem': True, 'runAsUser': 10001, 'runAsGroup': 10001, 'seLinuxOptions': None, 'seccompProfile': {'type': 'RuntimeDefault'}})

Section: Resources
- (None, 'resources', {'limits': {'cpu': '500m', 'memory': '1Gi'}, 'requests': {'cpu': '200m', 'memory': '512Mi'}})
- ('rules', 'resources', [])

Section: PodAnnotations
- (None, 'podAnnotations', {'container.apparmor.security.beta.kubernetes.io': 'runtime/default'})

Section: Volumes
- (None, 'volumes', [{'hostPath': None}])

Section: ExternalIPs
- (None, 'externalIPs', None)

Section: ServiceAccount
- (None, 'serviceAccount', {'anonymous-auth': False})

Section: ClusterRole
- (None, 'clusterRole', {'secrets': False, 'wildcard': False})

Section: Rules
- (None, 'rules', {'resources': [], 'verbs': ['list', 'watch', 'get']})

Section: Image
- (None, 

In [24]:
# Save updated YAML file
def save_yaml(file_path, data):
    with open(file_path, "w") as file:
        yaml.safe_dump(data, file)

In [25]:
def overwrite_sections(data, remediation_file):
    remediation_sections = identify_sections(remediation_file)
    
    # Iterate through all sections and overwrite data
    for section, remediation_values in remediation_sections.items():
        if remediation_values:
            for parent_key, key, remediation_value in remediation_values:
                # Update the corresponding key in the original data
                if parent_key and parent_key in data:
                    print(f"Updating {key} in section {section} under {parent_key} with remediation value.")
                    data[parent_key][key] = remediation_value
                elif key in data:
                    print(f"Updating {key} in section {section} with remediation value.")
                    data[key] = remediation_value
    return data


# Main processing
data = load_yaml(file_path)
with open("nested_resolutions.json", "r") as f:
    remediation_file = json.load(f)

print("\nOriginal Data Sections:")
print_sections(data)

print("\nRemediation Sections:")
print_sections(remediation_file)

# Overwrite sections
updated_data = overwrite_sections(data, remediation_file)

# Save the updated YAML file
updated_file_path = f"{remediated_charts_dir}/{subdirectory}/updated_values.yaml"
save_yaml(updated_file_path, updated_data)
print(f"\nUpdated values.yaml file saved to: {updated_file_path}")


Original Data Sections:
Identified Sections:

Section: SecurityContext
- ('application', 'securityContext', {})

Section: Resources
- ('application', 'resources', {})

Section: PodAnnotations
No occurrences found.

Section: Volumes
- ('deployment', 'volumes', [])

Section: ExternalIPs
No occurrences found.

Section: ServiceAccount
- (None, 'serviceAccount', {'create': True, 'annotations': {}, 'name': ''})

Section: ClusterRole
No occurrences found.

Section: Rules
No occurrences found.

Section: Image
- (None, 'image', {'repository': 'nginx', 'tag': '', 'pullPolicy': 'IfNotPresent'})

Section: HostSettings
No occurrences found.

Remediation Sections:
Identified Sections:

Section: SecurityContext
- (None, 'securityContext', {'allowPrivilegeEscalation': False, 'capabilities': {'drop': ['ALL'], 'add': []}, 'runAsNonRoot': True, 'readOnlyRootFilesystem': True, 'runAsUser': 10001, 'runAsGroup': 10001, 'seLinuxOptions': None, 'seccompProfile': {'type': 'RuntimeDefault'}})

Section: Resourc